In [127]:
!git clone https://github.com/abecode/teaching_reviews.git

fatal: destination path 'teaching_reviews' already exists and is not an empty directory.


In [128]:
!cp /content/teaching_reviews/data_prodigy/prodigy.sentence-level.20240701.db /content/
!cp /content/teaching_reviews/data_prodigy/prodigy.sentence-level.20240605.db /content/

In [129]:
!ls /content/

prodigy.sentence-level.20240605.db  sample_data
prodigy.sentence-level.20240701.db  teaching_reviews


In [130]:
import sqlite3

db_path = '/content/prodigy.sentence-level.20240701.db'


In [131]:
conn = sqlite3.connect(db_path)

In [132]:
sakthi_annotations = conn.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                 from dataset, example, link
                                 where dataset.id = link.dataset_id
                                   and link.example_id = example.id
                                   and dataset.name = 'teaching_reviews_sentences-sakthi';""").fetchall()

In [133]:
len(sakthi_annotations)

725

In [134]:
jenny_annotations = conn.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                 from dataset, example, link
                                 where dataset.id = link.dataset_id
                                   and link.example_id = example.id
                                   and dataset.name = 'teaching_reviews_sentences-jenny';""").fetchall()


In [135]:
len(jenny_annotations)

725

In [136]:
abe_annotations = conn.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                 from dataset, example, link
                                 where dataset.id = link.dataset_id
                                   and link.example_id = example.id
                                   and dataset.name = 'teaching_reviews_sentences-abe';""").fetchall()


In [137]:
len(abe_annotations)

725

In [138]:
mengyuan_annotations = conn.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                 from dataset, example, link
                                 where dataset.id = link.dataset_id
                                   and link.example_id = example.id
                                   and dataset.name = 'teaching_reviews_sentences-mengyuan';""").fetchall()

In [139]:
len(mengyuan_annotations)

725

In [140]:
input_hashes = [];
task_hashes = [];
for annotation in sakthi_annotations:
  input_hashes.append(annotation[5])
  task_hashes.append(annotation[6])
for annotation in jenny_annotations:
  input_hashes.append(annotation[5])
  task_hashes.append(annotation[6])

In [141]:
print(len(input_hashes))
print(len(task_hashes))
print(len(set(input_hashes)))
print(len(set(task_hashes)))

1450
1450
1450
725


In [142]:
import pandas as pd
import json
import numpy as np

In [143]:
sakthi_anno_df = pd.DataFrame({"annotator": ["sakthi" for x in sakthi_annotations],
                           "input_hash": [x[5] for x in sakthi_annotations],
                           "task_hash": [x[6] for x in sakthi_annotations],
                           #"content": [x[8] for x in sakthi_annotations],
                           "text": [json.loads(x[8])['text'] for x in sakthi_annotations],
                           "annotation":  [json.loads(x[8])['accept'] for x in sakthi_annotations]})

In [144]:
jenny_anno_df = pd.DataFrame({"annotator": ["jenny" for x in jenny_annotations],
                           "input_hash": [x[5] for x in jenny_annotations],
                           "task_hash": [x[6] for x in jenny_annotations],
                           #"content": [x[8] for x in jenny_annotations],
                           "text": [json.loads(x[8])['text'] for x in jenny_annotations],
                           "annotation":  [json.loads(x[8])['accept'] for x in jenny_annotations]})

In [145]:
abe_anno_df = pd.DataFrame({"annotator": ["abe" for x in abe_annotations],
                           "input_hash": [x[5] for x in abe_annotations],
                           "task_hash": [x[6] for x in abe_annotations],
                           #"content": [x[8] for x in abe_annotations],
                           "text": [json.loads(x[8])['text'] for x in abe_annotations],
                           "annotation":  [json.loads(x[8])['accept'] for x in abe_annotations]})

In [146]:
mengyuan_anno_df = pd.DataFrame({"annotator": ["mengyuan" for x in mengyuan_annotations],
                           "input_hash": [x[5] for x in mengyuan_annotations],
                           "task_hash": [x[6] for x in mengyuan_annotations],
                           #"content": [x[8] for x in mengyuan_annotations],
                           "text": [json.loads(x[8])['text'] for x in mengyuan_annotations],
                           "annotation":  [json.loads(x[8])['accept'] for x in mengyuan_annotations]})

In [147]:
merged_df_all_label = pd.DataFrame({
    "sakthi_annotation": sakthi_anno_df["annotation"],
    "jenny_annotation": jenny_anno_df["annotation"],
    "abe_annotation": abe_anno_df["annotation"],
    "mengyuan_annotation": mengyuan_anno_df["annotation"]
}).dropna()

merged_df_all_label

,sakthi_annotation,jenny_annotation,abe_annotation,mengyuan_annotation
0,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
1,[SUGGESTION],"[SUGGESTION, NEGATIVE]","[MIXED, SUGGESTION]",[SUGGESTION]
2,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
3,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
4,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
...,...,...,...,...
720,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
721,[POSITIVE],[POSITIVE],[POSITIVE],[POSITIVE]
722,[SUGGESTION],"[MIXED, NEGATIVE, SUGGESTION]","[NEGATIVE, SUGGESTION, MIXED]",[SUGGESTION]
723,[NEUTRAL],[SUGGESTION],[SUGGESTION],[POSITIVE]


In [148]:
 # Drop rows with None values if any
def map_labels(label):
    if label == []:
      print("found one")
      return ['UNSURE']
    else:
      return label
merged_df_all_label = merged_df_all_label.map(map_labels)

found one


Calculate the combinations among 6 labels. There is a fomula in math like 6C1 + 6C2 + ... 6C6 =

In [149]:
import math
total = 0
for i in range(1,7):
  # print(i)
  total += math.comb(6, i)

print("Total combinations: ", total)

Total combinations:  63


Label set: All the possible combinations

In [150]:
import itertools

all_choice = set()

labels = ['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'UNSURE', 'SUGGESTION']
for i in range(1,7):
  for a in list(itertools.combinations(labels, i)):
    a = list(a)
    a.sort()
    all_choice.add(str(a))

all_choice

{"['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'SUGGESTION', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'SUGGESTION']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'SUGGESTION', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'SUGGESTION']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'NEUTRAL']",
 "['MIXED', 'NEGATIVE', 'POSITIVE', 'SUGGESTION', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'POSITIVE', 'SUGGESTION']",
 "['MIXED', 'NEGATIVE', 'POSITIVE', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'POSITIVE']",
 "['MIXED', 'NEGATIVE', 'SUGGESTION', 'UNSURE']",
 "['MIXED', 'NEGATIVE', 'SUGGESTION']",
 "['MIXED', 'NEGATIVE', 'UNSURE']",
 "['MIXED', 'NEGATIVE']",
 "['MIXED', 'NEUTRAL', 'POSITIVE', 'SUGGESTION', 'UNSURE']",
 "['MIXED', 'NEUTRAL', 'POSITIVE', 'SUGGESTION']",
 "['MIXED', 'NEUTRAL', 'POSITIVE', 'UNSURE']",
 "['MIXED', 'NEUTRAL', 'POSITIVE']",
 "['MIXED', 

label all the possible combinations

In [151]:
mapdict = {}
for index, choice in enumerate(list(all_choice)):
  mapdict[choice]=index

mapdict

{"['NEGATIVE', 'NEUTRAL', 'SUGGESTION', 'UNSURE']": 0,
 "['NEUTRAL', 'POSITIVE', 'SUGGESTION', 'UNSURE']": 1,
 "['MIXED', 'NEUTRAL']": 2,
 "['UNSURE']": 3,
 "['NEGATIVE', 'NEUTRAL', 'POSITIVE', 'SUGGESTION']": 4,
 "['NEGATIVE', 'SUGGESTION']": 5,
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'SUGGESTION', 'UNSURE']": 6,
 "['MIXED', 'POSITIVE', 'UNSURE']": 7,
 "['SUGGESTION', 'UNSURE']": 8,
 "['POSITIVE', 'UNSURE']": 9,
 "['MIXED', 'NEGATIVE', 'POSITIVE']": 10,
 "['MIXED', 'NEGATIVE', 'UNSURE']": 11,
 "['NEGATIVE', 'POSITIVE', 'UNSURE']": 12,
 "['MIXED', 'NEUTRAL', 'SUGGESTION']": 13,
 "['POSITIVE']": 14,
 "['MIXED']": 15,
 "['NEGATIVE', 'SUGGESTION', 'UNSURE']": 16,
 "['SUGGESTION']": 17,
 "['NEGATIVE', 'POSITIVE']": 18,
 "['MIXED', 'NEGATIVE', 'POSITIVE', 'SUGGESTION', 'UNSURE']": 19,
 "['NEUTRAL', 'POSITIVE', 'SUGGESTION']": 20,
 "['MIXED', 'POSITIVE', 'SUGGESTION']": 21,
 "['MIXED', 'NEUTRAL', 'POSITIVE']": 22,
 "['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'SUGGESTION']": 23,
 "['

In [152]:
def map_labels(label):
    label.sort()
    label1 = mapdict.get(str(label), -1)
    if label1 == -1:
      print(label)
    return label1
df_numeric = merged_df_all_label.map(map_labels)

In [153]:
df_numeric

,sakthi_annotation,jenny_annotation,abe_annotation,mengyuan_annotation
0,14,14,14,14
1,17,5,56,17
2,14,14,14,14
3,14,14,14,14
4,14,14,14,14
...,...,...,...,...
720,14,14,14,14
721,14,14,14,14
722,17,61,61,17
723,25,17,17,14


In [154]:
n_categories = len(mapdict)
counts = np.zeros((len(df_numeric), n_categories))

for i in range(len(df_numeric)):
    counts[i] = np.bincount(df_numeric.iloc[i], minlength=n_categories)

print(counts)
print(len(counts), len(counts[0]))

print(counts[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
725 63
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [155]:
from statsmodels.stats.inter_rater import fleiss_kappa

kappa = fleiss_kappa(counts)
print(f'Fleiss\' kappa for the all labels: {kappa}')

Fleiss' kappa for the all labels: 0.6080284262742994
